<a href="https://colab.research.google.com/github/VavRe/nlp-ut/blob/main/CA5/mBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install sacremoses
!pip install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=d5fd5ed62fb89ab8d857ff2ef8617d1da5547ee6bb3818d4634d65a7d6c7dec0
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [2]:
import os
import json
import numpy as np
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import torch

In [3]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:  
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
BASE_PATH = "/content/drive/MyDrive/NLP/CA5"
TOKENIZED_DATA_PATH = os.path.join(BASE_PATH, "mbert_tokenized_data")
TRAIN_DATA_PATH = os.path.join(TOKENIZED_DATA_PATH, "train")
VALID_DATA_PATH = os.path.join(TOKENIZED_DATA_PATH, "valid")
TEST_DATA_PATH = os.path.join(TOKENIZED_DATA_PATH, "test")
DATA_DIR_PATH = os.path.join(BASE_PATH, "mbert_data_dir")

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
text = "This is a test"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
!mkdir -p ./data_bin

In [8]:
!fairseq-preprocess   --joined-dictionary	 --source-lang en --target-lang fa \
  --trainpref {TRAIN_DATA_PATH} \
  --validpref {VALID_DATA_PATH} \
  --testpref {TEST_DATA_PATH} \
  --destdir {DATA_DIR_PATH}

2023-06-10 16:03:30.763421: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-10 16:03:33 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-06-10 16:03:34 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, s

In [6]:
with open(os.path.join(DATA_DIR_PATH, "dict.en.txt"), "r") as f:
    mbert_dict = f.read().splitlines()

In [7]:
len(mbert_dict)

33212

In [8]:
mbert_dict_normalized = {}
for index,item in enumerate(mbert_dict):
    if item.split()[1].isdigit():
        mbert_dict_normalized[item.split()[0]] = item.split()[1]
    else:
        mbert_dict_normalized[item.split()[1]] = item.split()[0]

In [9]:
len(mbert_dict_normalized)

33212

In [10]:
import random
subset = dict(random.sample(mbert_dict_normalized.items(),100))
chosen_set = mbert_dict_normalized

<ipython-input-10-7ea466ab00f6>:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  subset = dict(random.sample(mbert_dict_normalized.items(),100))


In [12]:
len(chosen_set)

33212

In [13]:
mbert_dict_embeddings = {}

for key in tqdm(chosen_set.keys()):
    encoded = tokenizer.encode(key)
    input_ids = torch.tensor([encoded])

    with torch.no_grad():
      embedding = model(input_ids)[0].mean(1)[0].tolist()

    mbert_dict_embeddings[key] = embedding



100%|██████████| 33212/33212 [37:05<00:00, 14.92it/s]


In [14]:
final_list = []

for key,value in mbert_dict_embeddings.items():
  embd = ' '.join([str(x) for x in value])
  sample = f"{key} {embd}"
  final_list.append(sample)

In [15]:
with open(os.path.join(".","mbert_dict_embeddings_new.txt"), 'w') as f:
    f.write(str(len(chosen_set)))
    f.write(" ")
    f.write(str(len(list(chosen_set.values())[0])))
    f.write("\n")
    f.write("\n".join(final_list))


In [17]:
with open('./mbert_dict_embeddings_new.txt',"r") as f:
  dummy = f.readlines()

In [21]:
dummy[5924]

'Who -0.2196490615606308 -0.17202764749526978 1.3276394605636597 0.326817125082016 -0.939431369304657 0.06769527494907379 -0.34497570991516113 -0.42862558364868164 0.08538702875375748 0.14544257521629333 0.32078075408935547 0.24572698771953583 0.7457945942878723 -0.28421273827552795 -0.4417015314102173 -0.5618670582771301 -0.6064625382423401 1.151857852935791 -0.13414651155471802 -0.004638587590306997 0.1602405309677124 0.1520642191171646 -0.6476603150367737 0.1911998838186264 0.48194265365600586 0.0634564757347107 -0.5599460601806641 0.4333358705043793 0.08870700746774673 -0.462372750043869 0.02980154938995838 -0.15479086339473724 0.04918955639004707 0.19831854104995728 0.006706899497658014 0.027898654341697693 -0.7261903285980225 0.028216667473316193 0.29348206520080566 -0.5757322311401367 -0.49874162673950195 0.14376090466976166 -0.09991160780191422 0.29301905632019043 0.2846504747867584 0.16636912524700165 0.06830549985170364 0.04867468401789665 -0.02325158379971981 -0.629093647003

In [22]:
!fairseq-train \
  /content/drive/MyDrive/NLP/CA5/mbert_data_dir \
  --arch lstm --share-decoder-input-output-embed --share-all-embeddings --encoder-embed-dim 768 --decoder-embed-dim 768 --decoder-out-embed-dim 768 --encoder-embed-path /content/mbert_dict_embeddings_new.txt   \
  --optimizer adam --adam-betas '(0.9,0.98)' --clip-norm 0.0 \
  --lr 2.5e-3 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
  --dropout 0.25 --weight-decay 0.0001 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --max-tokens 4096 \
  --eval-bleu \
  --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
  --eval-bleu-detok moses \
  --eval-bleu-print-samples \
  --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
  --fp16 --memory-efficient-fp16 \
  --max-epoch 5 \
  --save-dir ./content/drive/MyDrive/mbert/checkpoints/ \
  --tensorboard-logdir ./content/drive/MyDrive/mbert/log/



2023-06-11 09:34:29.439029: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-11 09:34:30 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-11 09:34:31 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-06-11 09:34:33 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': './content/drive/MyDrive/mbert/log/', 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_

In [25]:
!fairseq-generate \
    /content/drive/MyDrive/NLP/CA5/mbert_data_dir \
    --batch-size 128 \
    --path "./content/drive/MyDrive/mbert/checkpoints/checkpoint_best.pt" \
    --beam 5 > "./content/drive/MyDrive/mbert/new_eval.txt"

2023-06-11 11:18:50.196899: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-11 11:18:54 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-06-11 11:18:57 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'a

In [18]:
!cp ./mbert_dict_embeddings_new.txt /content/drive/MyDrive/NLP/CA5/mbert_tokenized_data

In [24]:
!cp -r /content/content/drive/MyDrive/mbert /content/drive/MyDrive/mbert_new
!cp  /content/content/drive/MyDrive/mbert/new_eval.txt /content/drive/MyDrive/mbert_new/new_eval.txt

In [27]:
!fairseq-train \
  /content/drive/MyDrive/NLP/CA5/mbert_data_dir \
  --arch lstm --encoder-freeze-embed --decoder-freeze-embed --share-decoder-input-output-embed --share-all-embeddings --encoder-embed-dim 768 --decoder-embed-dim 768 --decoder-out-embed-dim 768 --encoder-embed-path /content/mbert_dict_embeddings_new.txt   \
  --optimizer adam --adam-betas '(0.9,0.98)' --clip-norm 0.0 \
  --lr 2.5e-3 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
  --dropout 0.25 --weight-decay 0.0001 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --max-tokens 4096 \
  --eval-bleu \
  --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
  --eval-bleu-detok moses \
  --eval-bleu-print-samples \
  --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
  --fp16 --memory-efficient-fp16 \
  --max-epoch 5 \
  --save-dir ./content/drive/MyDrive/mbert_frozen/checkpoints/ \
  --tensorboard-logdir ./content/drive/MyDrive/mbert_frozen/log/



2023-06-11 11:23:31.174400: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-11 11:23:33 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-11 11:23:33 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-06-11 11:23:36 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': './content/drive/MyDrive/mbert_frozen/log/', 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp

In [28]:
!fairseq-generate \
    /content/drive/MyDrive/NLP/CA5/mbert_data_dir \
    --batch-size 128 \
    --path "./content/drive/MyDrive/mbert_frozen/checkpoints/checkpoint_best.pt" \
    --beam 5 > "./content/drive/MyDrive/mbert_frozen/new_eval.txt"

2023-06-11 12:46:04.724955: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-11 12:46:07 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-06-11 12:46:11 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'a

In [29]:
!cp -r /content/content/drive/MyDrive/mbert_frozen /content/drive/MyDrive/mbert_new
